In [ ]:
pip install tensorflow

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

In [ ]:
# Define the architecture of the colorization model
def colorization_model(input_shape):
    # Encoder
    inputs = layers.Input(shape=input_shape)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(conv1)
    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', strides=2)(conv3)
    conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    # Decoder
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    up1 = layers.UpSampling2D((2, 2))(conv7)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
    up2 = layers.UpSampling2D((2, 2))(conv8)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up2)
    # Output
    output = layers.Conv2D(2, (3, 3), activation='tanh', padding='same')(conv9)
    return models.Model(inputs, output)

In [ ]:
# Prepare dummy grayscale and color images (replace with your data)
grayscale_images = np.random.rand(100, 128, 128, 1)
color_images = np.random.rand(100, 128, 128, 2)

In [ ]:
# Preprocess data (normalize pixel values)
grayscale_images = grayscale_images / 255.0
color_images = color_images / 255.0

In [ ]:
# Define model and compile
model = colorization_model((128, 128, 1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(grayscale_images, color_images, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 77s 23s/step - loss: 6.8644e-06 - val_loss: 2.0863e-06
Epoch 2/10
3/3 [==============================] - 74s 23s/step - loss: 2.7176e-06 - val_loss: 1.8656e-06
Epoch 3/10
3/3 [==============================] - 73s 23s/step - loss: 1.7963e-06 - val_loss: 1.8702e-06
Epoch 4/10
3/3 [==============================] - 73s 23s/step - loss: 1.6324e-06 - val_loss: 1.4931e-06
Epoch 5/10
3/3 [==============================] - 73s 23s/step - loss: 1.5000e-06 - val_loss: 1.4515e-06
Epoch 6/10
3/3 [==============================] - 78s 25s/step - loss: 1.4478e-06 - val_loss: 1.4918e-06
Epoch 7/10
3/3 [==============================] - 75s 24s/step - loss: 1.4307e-06 - val_loss: 1.3553e-06
Epoch 8/10
3/3 [==============================] - 72s 23s/step - loss: 1.3800e-06 - val_loss: 1.3338e-06
Epoch 9/10
3/3 [==============================] - 73s 23s/step - loss: 1.3448e-06 - val_loss: 1.3490e-06
Epoch 10/10
3/3 [==============================] - 72s 

In [ ]:
# Evaluate the model
loss = model.evaluate(grayscale_images, color_images)
print("Test Loss:", loss)

4/4 [==============================] - 24s 5s/step - loss: 1.3289e-06
Test Loss: 1.3289350135892164e-06


In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the model
model.save("colorization_model.h5")
model.save("/content/drive/My Drive/MyProject/model/colorization_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Predict colorized images
predicted_color_images = model.predict(grayscale_images)

4/4 [==============================] - 25s 6s/step


In [ ]:
# Convert color images and predicted color images to the range [0, 255]
color_images = color_images * 255
predicted_color_images = predicted_color_images * 255

In [ ]:
# Flatten the arrays for calculating metrics
color_images_flat = color_images.flatten()
predicted_color_images_flat = predicted_color_images.flatten()

In [ ]:
# Calculate MAE
mae = mean_absolute_error(color_images_flat, predicted_color_images_flat)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(color_images_flat, predicted_color_images_flat))

# Calculate PSNR
psnr = tf.image.psnr(color_images, predicted_color_images, max_val=255)

In [ ]:
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr.numpy())

Mean Absolute Error (MAE): 0.253040082344735
Root Mean Squared Error (RMSE): 0.2939625943776313
Peak Signal-to-Noise Ratio (PSNR): [58.773293 58.74341  58.761627 58.749493 58.74262  58.764946 58.776657
 58.79003  58.743786 58.793373 58.748947 58.80185  58.75482  58.757862
 58.77578  58.772495 58.75224  58.77256  58.785492 58.77853  58.734993
 58.766407 58.755386 58.72138  58.747124 58.79575  58.77038  58.73134
 58.757626 58.763012 58.756264 58.733753 58.836384 58.77149  58.748
 58.74655  58.7525   58.798462 58.76082  58.81426  58.752747 58.766655
 58.72019  58.74447  58.80323  58.76433  58.76829  58.7994   58.791183
 58.76862  58.7792   58.78246  58.77178  58.75642  58.766735 58.78392
 58.735092 58.760857 58.785007 58.782692 58.75946  58.770683 58.771202
 58.75492  58.77906  58.73055  58.741592 58.76896  58.74136  58.76874
 58.72013  58.784092 58.78478  58.798653 58.782433 58.766468 58.78508
 58.752674 58.75086  58.7723   58.72191  58.769794 58.80149  58.74279
 58.73623  58.76896  58.7

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import cv2

# Load the trained model from the .h5 file
model = load_model('your_model.h5')

# Save the model architecture to a .prototxt file
with open('model.prototxt', 'w') as f:
    f.write(model.to_json())

# Save the model weights to a .caffemodel file
model.save_weights('model.caffemodel')

# Generate pts_in_hull.npy file (example)
# You need to replace this with actual data generation logic
pts_in_hull = np.random.rand(313, 2)  # Example random points
np.save('pts_in_hull.npy', pts_in_hull)

In [8]:
# import numpy as np
# from tensorflow.keras.models import load_model
# import os
# from tensorflow.keras.optimizers import Adam

# # Function to generate .prototxt and .caffemodel files from a Keras model
# def generate_prototxt_and_caffemodel(model, folder_path):
#     # Convert Keras model to .prototxt
#     prototxt_path = os.path.join(folder_path, 'generator.prototxt')
#     caffemodel_path = os.path.join(folder_path, 'generator.caffemodel')
#     with open(prototxt_path, 'w') as f:
#         f.write(model.to_json())

#     # Save model weights to .caffemodel
#     model.save_weights(caffemodel_path)

# # Function to generate pts_in_hull.npy (replace with actual logic)
# def generate_pts_in_hull(pts_in_hull_path):
#     # Example random points generation (replace with actual logic)
#     pts_in_hull = np.random.rand(313, 2)
#     np.save(pts_in_hull_path, pts_in_hull)

# # Load the generator model
# generator_model = load_model('/content/drive/MyDrive/MyProject/model_v1/generator_epoch_150.h5')

# # Compile the generator model manually
# generator_model.compile(optimizer=Adam(), loss='mse')

# # Generate .prototxt and .caffemodel files for the generator
# generate_prototxt_and_caffemodel(generator_model, '/content/drive/MyDrive/MyProject/model_v1')

# # Generate pts_in_hull.npy for the generator
# generate_pts_in_hull('/content/drive/MyDrive/MyProject/model_v1/pts_in_hull.npy')
